In [ ]:
# حذف بسته‌های مشکل‌ساز
!pip uninstall -y numpy pandas torch transformers datasets hazm tokenizers accelerate

# نصب نسخه‌هایی که با Colab و transformers سازگار و پایدار باشند
%pip install numpy==1.26.2 pandas==2.2.2
%pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
%pip install datasets==2.14.0 tokenizers==0.15.2
%pip install transformers==4.41.0
%pip install hazm accelerate

In [ ]:
try:
    import numpy as np
    import pandas as pd
    import torch
    from hazm import Normalizer
    from datasets import Dataset
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

    print("✅ کتابخانه‌ها با موفقیت import شدند!")
    print(f"🔹 numpy: {np.__version__}")
    print(f"🔹 pandas: {pd.__version__}")
    print(f"🔹 torch: {torch.__version__}")
    print(f"🔹 transformers: {transformers.__version__}")

except Exception as e:
    print("❌ خطا در import:", e)


In [ ]:
import pandas as pd

# لینک‌ها
url1 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part1.csv"
url2 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part2.csv"

# 📥 خواندن فایل‌ها با تنظیمات دقیق
df1 = pd.read_csv(url1, sep="\t", header=0)
df2 = pd.read_csv(url2, sep="\t", header=0)

# 🔗 ادغام
df = pd.concat([df1, df2], ignore_index=True)

# حذف ستون‌های اضافه‌ای که اشتباه وارد شدن
expected_columns = ['comment_normalized', 'hate', 'spam', 'obscene', 'class']
df = df[expected_columns]

# نمایش اولیه
print(f"🔢 تعداد کل داده‌ها: {len(df)}")
print("🧾 ستون‌ها:", df.columns.tolist())
print(df.head(3))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# حذف ردیف‌هایی که مقدار کلاس ندارن
df = df.dropna(subset=["class"])

# دوباره لیبل‌ها رو بساز
label_encoder = LabelEncoder()
df.loc[:, "label"] = label_encoder.fit_transform(df["class"])

# نمایش کلاس‌ها
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("🔢 کلاس‌ها:", label_map)

# تبدیل به Dataset
dataset = Dataset.from_pandas(df[["comment_normalized", "label"]])
print("✅ Dataset آماده است!")

In [ ]:
from hazm import Normalizer
from transformers import AutoTokenizer

normalizer = Normalizer()
model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)س

def preprocess_data(texts):
    return tokenizer([normalizer.normalize(t) for t in texts], truncation=True, padding=True)

print("✅ توکنایزر و نرمالایزر آماده است!")


In [ ]:

encodings = preprocess_data(df['text'].tolist())
labels = df['label'].tolist()

dataset = Dataset.from_dict({
    'input_ids': encodings['input_ids'],
    'attention_mask': encodings['attention_mask'],
    'labels': labels
})

splits = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits['train']
eval_dataset = splits['test']

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./phicad_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    logging_dir='./phicad_logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

model.save_pretrained("phicad_model")
tokenizer.save_pretrained("phicad_model")

In [ ]:
def is_inappropriate(text):
    t = normalizer.normalize(text)
    inputs = tokenizer(t, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    return torch.argmax(logits, dim=1).item() == 1

comments = [
    "لعنت به این وضعیت خرابات!",
    "خیلی ممنون از خدمات شما"
]
for c in comments:
    print(c, "→", "نامناسب" if is_inappropriate(c) else "مناسب")